In [ ]:
!pip install welly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
import zipfile
from welly import Project
from IPython.display import HTML, display
import welly
import welly.quality as wq
import matplotlib.pyplot as plt

In [ ]:
zip_file = 'SPE.zip'

extract_folder = 'SPE'

# Распаковываем zip-файл
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [ ]:
wells = Project.from_las('SPE/*.LAS')

208it [00:15, 13.60it/s]


In [ ]:
df = wells.df()

/usr/local/lib/python3.11/dist-packages/welly/curve.py:470: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  if self.df.index.is_numeric() and not self.df.index.empty:
/usr/local/lib/python3.11/dist-packages/welly/curve.py:470: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  if self.df.index.is_numeric() and not self.df.index.empty:
/usr/local/lib/python3.11/dist-packages/welly/curve.py:470: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  if self.df.index.is_numeric() and not self.df.index.empty:
/usr/local/lib/python3.11/dist-packages/welly/curve.py:470: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  if self.df.index.is_numeric() and not self.df.index.empty:
/usr/local/lib/python3.11/dist-packages/welly/curve.py:470: FutureWarning: Index.is_numeric is deprecated. U

In [ ]:
# Расчет частот каротажей в скважинах без поностью пропущенных значений
count_logs = {}
for well in df.index.get_level_values(0).unique():
  for colum in df.columns:
    if df.loc[well][colum].notnull().any() == True:
      if colum in count_logs:
            count_logs[colum] += 1
      else:
            count_logs[colum] = 1
result_dict = {key: value / 208 for key, value in count_logs.items()}
dict(sorted(result_dict.items(), key = lambda x: x[1], reverse = True))

{'GR': 1.0,
 'ILD': 0.9951923076923077,
 'NPHI': 0.9230769230769231,
 'DPHI': 0.875,
 'CALI': 0.40384615384615385,
 'DELT': 0.0673076923076923,
 'RHOB': 0.052884615384615384,
 'PHID': 0.009615384615384616,
 'PHID_ORIG': 0.004807692307692308,
 'PHIN': 0.004807692307692308}

In [ ]:
df.head()

ILD   DPHI   NPHI       GR     CALI  RHOB  DELT  \
UWI                                                                             
00/07-08-067-02W4/0 345.00  3.126  0.218  0.494  105.703  172.531   NaN   NaN   
                    345.25  3.184  0.231  0.517  101.298  173.331   NaN   NaN   
                    345.50  3.213  0.236  0.512  104.025  174.031   NaN   NaN   
                    345.75  3.267  0.236  0.497  106.147  174.427   NaN   NaN   
                    346.00  3.310  0.235  0.467  103.677  174.922   NaN   NaN   

                            PHID  PHID_ORIG  PHIN  
UWI                                                
00/07-08-067-02W4/0 345.00   NaN        NaN   NaN  
                    345.25   NaN        NaN   NaN  
                    345.50   NaN        NaN   NaN  
                    345.75   NaN        NaN   NaN  
                    346.00   NaN        NaN   NaN

In [ ]:
df = df.drop(['DELT', 'PHID', 'PHID_ORIG', 'PHIN', 'RHOB', 'CALI'], axis = 1)

In [ ]:
df.index.names = ['UWI', 'DEPT']

In [ ]:
df.head()

ILD   DPHI   NPHI       GR
UWI                 DEPT                                
00/07-08-067-02W4/0 345.00  3.126  0.218  0.494  105.703
                    345.25  3.184  0.231  0.517  101.298
                    345.50  3.213  0.236  0.512  104.025
                    345.75  3.267  0.236  0.497  106.147
                    346.00  3.310  0.235  0.467  103.677

In [ ]:
# Отбор скважин, в которых каротажи не имеют полностью пропущенных значений
for well in df.index.get_level_values(0).unique():
    for colum in df.columns:
      if df.loc[well][colum].notnull().any() != True:
        df = df.drop(index = well, level = 'UWI')
        break

In [ ]:
wells = ['00/11-25-078-03W5/0', '00/02-29-073-15W4/0', '00/03-11-080-08W400', '00/10-01-071-11W4/0', '00/03-06-067-11W4/0', '00/05-20-080-13W4/0', 'AA/11-15-084-07W4/0', '00/07-12-075-13W4/0',
         '00/15-21-084-03W400', '00/04-21-074-05W5/0', '00/08-02-080-13W4/0', 'AA/10-14-084-19W400', '00/12-14-075-08W4/0', '00/10-35-080-20W4/0', '00/11-07-070-01W5/0', 'AA/12-29-083-10W400',
         '00/08-32-069-06W4/0', '02/10-30-072-14W4/0', '00/11-21-069-17W4/0', '00/11-05-085-06W400', 'AA/10-08-092-15W400', '00/06-06-080-12W4/0', '00/10-14-073-10W4/0', '00/07-34-067-08W4/0',
         '00/06-03-075-10W4/0', '00/10-10-071-15W4/0', '00/07-04-077-04W5/0', 'AA/06-23-077-25W4/0', '00/10-03-084-22W4/0', 'AA/06-14-076-07W4/0', '00/10-22-071-05W5/0', '00/05-06-077-02W4/0',
         '00/10-27-079-01W5/0', '00/14-14-079-22W4/0', '00/10-01-073-09W4/0', '00/07-32-077-22W4/0', '00/11-12-096-18W400', '00/03-09-075-23W4/0', 'AA/10-08-081-14W400', '00/04-29-076-23W4/0',
         '00/11-10-071-13W4/0', '00/07-09-077-04W5/0', '00/08-22-067-12W4/2', '00/11-04-075-06W4/0', '00/05-01-081-23W4/0', '00/06-05-076-04W5/0', '00/02-26-072-13W4/0', '00/03-19-068-09W4/0',
         '00/10-07-067-04W4/0', '00/06-24-081-23W4/0', '00/07-01-077-03W5/0', '00/16-21-074-24W4/0', '00/16-33-070-15W4/0', '00/11-03-071-01W5/0', '00/10-25-075-15W4/0', '00/12-22-078-02W5/0',
         '00/03-34-078-05W4/0', '00/07-33-074-11W4/0', '00/09-11-067-02W4/0', '00/09-09-069-09W4/0', '00/09-35-070-06W4/0', '00/15-34-073-25W4/0', '00/11-08-072-01W5/0', '00/10-34-069-15W4/0',
         '00/06-14-083-16W400', '00/10-05-080-15W4/0', '00/01-27-079-23W4/0', '00/06-07-069-02W4/0', '00/10-15-074-06W4/0', '00/12-36-070-16W4/0', '00/10-30-069-11W4/0', '00/05-04-069-04W4/0'
]

In [ ]:
df = df.loc[wells]

In [ ]:
df.head()

ILD   DPHI   NPHI       GR
UWI                 DEPT                                 
00/11-25-078-03W5/0 347.472  2.436  0.252  0.684   97.980
                    347.722  2.432  0.268  0.669  101.246
                    347.972  2.472  0.261  0.638  103.621
                    348.222  2.512  0.254  0.618  106.289
                    348.472  2.531  0.236  0.613  105.680

In [ ]:
tests = {'ILD': [wq.no_flat,
                  wq.no_gaps,
                  wq.all_positive],
         'DPHI': [wq.no_flat,
                  wq.no_gaps,
                  wq.all_between(-0.5,1)],
         'NPHI': [wq.no_flat,
                  wq.no_gaps,
                  wq.all_between(0,1)],
         'GR': [wq.no_flat,
                  wq.no_gaps,
                  wq.all_positive],}

In [ ]:
for well in df.index.get_level_values(0).unique():
  w = df.loc[well]
  wl = welly.Well({})
  for log in tests.keys():  # Проходим по логам, для которых есть тесты
        if log in w.columns:  # Проверяем, имеются ли данные по этому логe
            wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)

 # show test results in HTML table
  print('\n', '\n')
  print('WELL NAME: ' + well)
  print('CURVE QUALITY SUMMARY')
  html = wl.qc_table_html(tests)
  display(HTML(html, metadata = {'title': well,}))


 

WELL NAME: 00/11-25-078-03W5/0
CURVE QUALITY SUMMARY


<ipython-input-16-a7c7d625a172>:8: DeprecationWarning: In a future release, the basis argument will be removed; use index instead.
  wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)


Curve,Passed,Score,all_positive,no_flat,all_between,no_gaps
ILD,3 / 3,1.000,True,True,,True
DPHI,3 / 3,1.000,,True,True,True
NPHI,3 / 3,1.000,,True,True,True
GR,3 / 3,1.000,True,True,,True



 

WELL NAME: 00/02-29-073-15W4/0
CURVE QUALITY SUMMARY


<ipython-input-16-a7c7d625a172>:8: DeprecationWarning: In a future release, the basis argument will be removed; use index instead.
  wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)


...


 

WELL NAME: 00/10-30-069-11W4/0
CURVE QUALITY SUMMARY


Curve,Passed,Score,all_positive,no_flat,all_between,no_gaps
ILD,3 / 3,1.000,True,True,,True
DPHI,3 / 3,1.000,,True,True,True
NPHI,3 / 3,1.000,,True,True,True
GR,3 / 3,1.000,True,True,,True


<ipython-input-16-a7c7d625a172>:8: DeprecationWarning: In a future release, the basis argument will be removed; use index instead.
  wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)



 

WELL NAME: 00/05-04-069-04W4/0
CURVE QUALITY SUMMARY


Curve,Passed,Score,all_positive,no_flat,all_between,no_gaps
ILD,3 / 3,1.000,True,True,,True
DPHI,3 / 3,1.000,,True,True,True
NPHI,3 / 3,1.000,,True,True,True
GR,3 / 3,1.000,True,True,,True


In [ ]:
df_plot = df.copy()
df_plot.reset_index(inplace=True)

In [ ]:
def create_plot(wellname, dataframe,
                curves_to_plot, depth_curve,
                log_curves=[]):

    # Count the number of tracks we need
    num_tracks = len(curves_to_plot)

    # Setup the figure and axes
    fig, ax = plt.subplots(nrows=1, ncols=num_tracks, figsize=(num_tracks*2, 10))

    # Create a super title for the entire plot
    fig.suptitle(wellname, fontsize=20, y=1.05)

    # Loop through each curve in curves_to_plot and create a track with that data
    for i, curve in enumerate(curves_to_plot):


        ax[i].plot(dataframe[curve], depth_curve)

        # Setup a few plot cosmetics
        ax[i].set_title(curve, fontsize=14, fontweight='bold')
        ax[i].grid(which='major', color='lightgrey', linestyle='-')

        # We want to pass in the deepest depth first, so we are displaying the data
        # from shallow to deep
        ax[i].set_ylim(depth_curve.max(), depth_curve.min())

        # Only set the y-label for the first track. Hide it for the rest
        if i == 0:
            ax[i].set_ylabel('DEPTH (m)', fontsize=18, fontweight='bold')
        else:
            plt.setp(ax[i].get_yticklabels(), visible = False)

        # Check to see if we have any logarithmic scaled curves
        if curve in log_curves:
            ax[i].set_xscale('log')
            ax[i].grid(which='minor', color='lightgrey', linestyle='-')

    plt.tight_layout()
    plt.show()

In [ ]:
grouped = df_plot.groupby('UWI')
dfs_wells = []
wellnames = []

In [ ]:
for well, data in grouped:
  dfs_wells.append(data)
  wellnames.append(well)
curves_to_plot = ['ILD', 'GR', 'NPHI', 'DPHI']

In [ ]:
for well in range(len(wellnames)):
  create_plot(wellnames[well], dfs_wells[well],
              curves_to_plot, dfs_wells[well]['DEPT'])

In [ ]:
df.to_csv('SPE_clear.csv', index = True)